# Q-Learning (Q-table)
 Erster Versuch einer Implementierung von Q-Learning
### Mithilfe des Tutorials von https://www.datacamp.com/tutorial/introduction-q-learning-beginner-tutorial

 Die wichtigsten Begriffe:
 - Environment  : Die Umgebung, welche auf die Aktionen des Agent reagiert
 - Agent        : An entity that acts and operates within an environment
 - State        : Der momentane Zustand des Environment
 - Reward       : Die Rückmeldung des Environment darauf, ob eine Action gute oder schlechte Resultate erbracht hat (reward/penalty)
 - Action       : Eine vom Agent ausgeführte Aktion, welche dann Einfluss auf das Environment und die weiteren Actions hat

 ![Reinforcement Learning](pics\RL_overview.webp)
 

## Install, initialize and import stuff

In [5]:
%%capture 
%pip install pyglet==1.5.1
%pip install PyOpenGL   # %apt install python-opengL
%pip install imageio[ffmpeg]    #%apt install ffmpeg
# %apt install xvfb
%pip3 install pyvirtualdisplay


In [6]:
# use this if used with WSL (for gif)
# import os
# from pyvirtualdisplay import Display
# virtual_display = Display(visible=True, size=(1400,900))
# os.environ["DISPLAY"] = "localhost:0.0"  # Use "localhost:0.0" for Xming
# virtual_display.start()


In [7]:
%pip install gym==0.24
%pip install pygame
%pip install numpy
%pip install imageio imageio_ffmpeg
%pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import numpy as np
import gym
import random
import imageio
from tqdm.notebook import trange

## Actual code

In [9]:
env = gym.make("FrozenLake-v1",map_name="4x4",is_slippery=False)

print("Observation Space", env.observation_space) # prints out the observation space, which is the set of all possible states that the agent can observe in the environment.
print("Sample observation", env.observation_space.sample()) #  a randomly generated state from the observation space

Observation Space Discrete(16)
Sample observation 10


In [10]:
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())

Action Space Shape 4
Action Space Sample 0


### Create and Initialize the Q-table


In [11]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

Qtable_frozenlake = np.zeros((state_space, action_space)) # create a 16 x 4 Array for the q-table


There are  16  possible states
There are  4  possible actions


### Policies

In [12]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  """
  acting policy
  1. Generates random number between 0 & 1
  2. if number greater than epsilon -> exploitation (action with highest value to the current state) 
  3. else -> exploration (random action)
  """
  random_int = random.uniform(0,1)
  if random_int > epsilon:
    action = np.argmax(Qtable[state])
  else:
    action = env.action_space.sample()
  return action

def greedy_policy(Qtable, state):
  """
  updating policy
  """
  action = np.argmax(Qtable[state]) # the action that the agent should take in order to maximize its reward
  return action

### Hyperparameters

In [13]:
# Hyperparameters, finetuning will give better results

# Training parameters
n_training_episodes = 10000
learning_rate = 0.7        

# Evaluation parameters
n_eval_episodes = 100      

# Environment parameters
env_id = "FrozenLake-v1"   
max_steps = 99             
gamma = 0.95               
eval_seed = []             

# Exploration parameters
max_epsilon = 1.0           
min_epsilon = 0.05           
decay_rate = 0.0005           

### Model Training

In [14]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in trange(n_training_episodes):
 
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Reset the environment
    state = env.reset()
    step = 0
    done = False

    # repeat
    for step in range(max_steps):
   
      action = epsilon_greedy_policy(Qtable, state, epsilon)

   
      new_state, reward, done, info = env.step(action)

   
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # If done, finish the episode
      if done:
        break
     
      # Our state is the new state
      state = new_state
  return Qtable

In [18]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html